<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#CTR-prediction" data-toc-modified-id="CTR-prediction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>CTR-prediction</a></span><ul class="toc-item"><li><span><a href="#Problem-Formulation" data-toc-modified-id="Problem-Formulation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Problem Formulation</a></span></li><li><span><a href="#Dataset" data-toc-modified-id="Dataset-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Dataset</a></span><ul class="toc-item"><li><span><a href="#Dataset-construction:" data-toc-modified-id="Dataset-construction:-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Dataset construction:</a></span></li><li><span><a href="#Format:" data-toc-modified-id="Format:-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Format:</a></span></li></ul></li><li><span><a href="#Metrics" data-toc-modified-id="Metrics-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Metrics</a></span></li></ul></li><li><span><a href="#Dataset-preprocessing" data-toc-modified-id="Dataset-preprocessing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dataset preprocessing</a></span><ul class="toc-item"><li><span><a href="#ML-Pipelines-(Transformers,-Estimators)" data-toc-modified-id="ML-Pipelines-(Transformers,-Estimators)-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline-components" target="_blank">ML Pipelines (Transformers, Estimators)</a></a></span><ul class="toc-item"><li><span><a href="#Prepare-stages-of-pipeline" data-toc-modified-id="Prepare-stages-of-pipeline-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Prepare stages of pipeline</a></span></li><li><span><a href="#Fit-and-save-pipeline" data-toc-modified-id="Fit-and-save-pipeline-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Fit and save pipeline</a></span></li><li><span><a href="#Load-fitted-pipeline" data-toc-modified-id="Load-fitted-pipeline-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Load fitted pipeline</a></span></li><li><span><a href="#Transform-dataset-using-pipeline" data-toc-modified-id="Transform-dataset-using-pipeline-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Transform dataset using pipeline</a></span></li><li><span><a href="#Make-dataset-split" data-toc-modified-id="Make-dataset-split-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Make dataset split</a></span></li></ul></li></ul></li><li><span><a href="#Classification" data-toc-modified-id="Classification-3"><span class="toc-item-num">3&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-classification-regression.html" target="_blank">Classification</a></a></span><ul class="toc-item"><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression" target="_blank">Logistic Regression</a></a></span><ul class="toc-item"><li><span><a href="#Define-and-Train-model" data-toc-modified-id="Define-and-Train-model-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Define and Train model</a></span></li></ul></li></ul></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html" target="_blank">Evaluation</a></a></span><ul class="toc-item"><li><span><a href="#Binary-classification-metrics" data-toc-modified-id="Binary-classification-metrics-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html#binary-classification" target="_blank">Binary classification metrics</a></a></span></li><li><span><a href="#Make-submission" data-toc-modified-id="Make-submission-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Make submission</a></span></li></ul></li></ul></div>

# CTR-prediction

## Problem Formulation

$\newcommand{\vecw}{{\bf w}}$
$\newcommand{\vecx}{{\bf x}}$

* Dataset: $X^N = \{ z_i \}^N_{i=1}$, где $z_i = (\vecx_i, y_i) \sim P(z), y_i \in \{0,1\}$
* Prediction: $$ \hat{y}_i = f_{\vecw}(\vecx_i) =  \mathbb{P} \left\{ y = 1 \mid \vecx_i \right\} $$
* Loss function (Binary Cross-Entropy): $$ \min\limits_{\vecw} \quad \frac{\lambda}{2}\| \vecw \|^2_2 - \frac{1}{N} \sum\limits_{i=1}^{N} y_i \log \hat{y}_i + (1-y_i) \log(1-\hat{y}_i) $$

## Dataset
$ $
<details>
  <summary>Click here to see the details</summary>

For more details see `/data/criteo/readme.txt`

### Dataset construction:


>There are 13 features taking **integer** values and 26
**categorical** features. The values of the categorical features have been hashed
onto 32 bits for anonymization purposes. 
Some features may have missing values.

> The rows are chronologically ordered by `id` column.

> The test set corresponds to events on the day following the training period. 
The first column (`label`) has been removed.


### Format:

> The columns are comma separeted with the following schema:
`<label>,<integer feature 1>, ... <integer feature 13>,<categorical feature 1>, ... <categorical feature 26>,<id>`

> When a value is missing, the field is "". There is no `label` field in the test set.

</details>
    
## Metrics

The evaluation metrics for this task are
* ROC AUC
* LogLoss
* [Normalized Entropy](https://quinonero.net/Publications/predicting-clicks-facebook.pdf)

In [1]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import os
import sys
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import pyspark
import pyspark.sql.functions as F
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import Row


spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("spark_sql_examples") \
    .config("spark.executor.memory", "20g") \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [162]:
DATA_PATH = '/workspace/data/criteo'

TRAIN_PATH = os.path.join(DATA_PATH, 'train.csv')

In [163]:
!ls /workspace/data/criteo

test.csv  train.csv


Lets begin our introduction to Spark [MLlib](https://spark.apache.org/docs/latest/ml-guide.html)

---
# Dataset preprocessing

Before we can train any prediction model on our dataset we need to conver each row into real-valued features vector ($\vecx \in \mathbb{R}^n$).

Spark MLlib provides easy to use tools for preprocessing raw features and turning them into suitable format.

In [164]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TRAIN_PATH)

In [165]:
df = df.sample(False, 0.5)

In [166]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (

Keeping only first two categorical features for simplicity

In [167]:
num_columns = ['_c{}'.format(i) for i in range(1, 14)]
cat_columns = ['_c{}'.format(i) for i in range(14, 40)][:2]
len(num_columns), len(cat_columns)

(13, 2)

In [168]:
df = df.fillna(0, subset=num_columns)

## [ML Pipelines (Transformers, Estimators)](https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline-components)


MLlib standardizes APIs for machine learning algorithms to make it easier to combine multiple algorithms into a single pipeline, or workflow.

* `Transformer`: A Transformer is an algorithm which can transform one DataFrame into another DataFrame. E.g., an ML model is a Transformer which transforms a DataFrame with features into a DataFrame with predictions.


* `Estimator`: An Estimator is an algorithm which can be fit on a DataFrame to produce a Transformer. E.g., a learning algorithm is an Estimator which trains on a DataFrame and produces a model.


* `Pipeline`: A Pipeline chains multiple Transformers and Estimators together to specify an ML workflow.

---
Basically speaking `transformer` is an instance of class that implements `transform` method, and both `estimator` and `pipeline` implements `transform` and `fit` methods.

---

### Prepare stages of pipeline

We might benefit from using `StringIndexer, OneHotEncoderEstimator, VectorAssembler` (see [doc](https://spark.apache.org/docs/latest/ml-features) for details) 

In [169]:
orig_df = df

In [170]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler

df = orig_df

stages = []

for i in cat_columns:
    stages.append(StringIndexer(inputCol=i, outputCol=(i + '-indexed'), handleInvalid='keep'))

indexed_cols = list(map(lambda c: c + '-indexed', cat_columns))
one_hot_cols = list(map(lambda c: c + '-one_hot', cat_columns))

stages.append(OneHotEncoderEstimator(inputCols=indexed_cols, outputCols=one_hot_cols, handleInvalid='keep'))
stages.append(VectorAssembler(inputCols=(num_columns + one_hot_cols), outputCol='features', handleInvalid='keep'))

pipeline = Pipeline(stages=stages)
pipeline_model = pipeline.fit(df)

### Fit and save pipeline

In [171]:
!mkdir -p /workspace/data/criteo_models

In [172]:
from pyspark.ml import PipelineModel

model_path = '/workspace/data/criteo_models/mllib_practice_model'

pipeline.fit(df).write().overwrite().save(model_path)

### Load fitted pipeline

In [173]:
from pyspark.ml import PipelineModel

pipeline_model = PipelineModel.load(model_path)

### Transform dataset using pipeline

In [174]:
df = pipeline_model.transform(df)

# we don't need any other colums anymore
df = df.select(['_c0', 'id', 'features']).withColumnRenamed('_c0', 'label')

In [175]:
df.head()

Row(label=1, id=26, features=SparseVector(1960, {1: 1.0, 2: 20.0, 3: 16.0, 4: 1548.0, 5: 93.0, 6: 42.0, 7: 32.0, 8: 912.0, 10: 15.0, 11: 1.0, 12: 16.0, 16: 1.0, 1448: 1.0}))

### Make dataset split

Spark provides [randomSplit](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.randomSplit) method.

It is not the best choice in our task since we have chronological order in data.

We need to implement our own split function which will split the data in parts with respect to chronological order.

In [176]:
# Это на самом деле скопировал с прошлой практики. Ожидалось что-то более умное?

from pyspark.sql.window import Window
import pyspark.sql.functions as F

def split_by_col(df, split_col, parts_fractions):
    """
    df - DataFrame
    split_col - total order column
    parts_fractions - fractions of resulting parts
    """
    
    N = df.count()
    n_train = parts_fractions[0] * N
    n_val = n_train + parts_fractions[1] * N

    row_window = Window.orderBy(split_col)

    tmp_df = df \
        .sort(split_col) \
        .withColumn('row_number',  F.row_number().over(row_window))
    
    train = tmp_df.filter(F.col('row_number') < n_train)
    val = tmp_df.filter(F.col('row_number') > n_train).filter(F.col('row_number') < n_val)
    test = tmp_df.filter(F.col('row_number') >= n_val)
    
    return (train, val, test)

In [177]:
train_df, val_df, test_df = split_by_col(df, 'id', [0.8, 0.1, 0.1])

In [178]:
N = df.count()

In [179]:
train_df.count() / N, val_df.count() / N, test_df.count() / N

(0.7999994547154842, 0.09999972735774208, 0.10000027264225791)

---
# [Classification](https://spark.apache.org/docs/latest/ml-classification-regression.html)

## [Logistic Regression](https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression)

### Define and Train model

In [180]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(regParam=0.2)
lr_model = lr.fit(train_df)

---
# [Evaluation](https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html)

## [Binary classification metrics](https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html#binary-classification)

* ROC AUC
* LogLoss
* Normalized Entropy

In [211]:
from pyspark.sql.types import *
from pyspark.mllib.evaluation import BinaryClassificationMetrics
import math

def rocauc(model, df):
    return model.evaluate(df).areaUnderROC


def logloss(model, df):
    EPS = 1e-9
    df = model.transform(df)
    
    def helper(l, p):
        if l == 1:
            return -math.log(p[1] + EPS)
        else:
            return -math.log(p[0] + EPS)

    helper_udf = F.udf(helper, FloatType())
    
    s = df \
        .withColumn('log_loss', helper_udf('label', 'probability')) \
        .select('log_loss') \
        .rdd.map(lambda r: r[0]).sum()

    return s / df.count()

def ne(model, df):
    lloss = logloss(model, df)
    p = df \
        .select('label') \
        .rdd.map(lambda r: r[0]).sum()
    p = p / df.count()
    return -lloss / (p * math.log(p) + (1 - p) * math.log(1 - p))

In [212]:
logloss(lr_model, val_df)

0.5394826954955808

In [156]:
rocauc(lr_model, val_df)

0.7010569136850341

In [213]:
ne(lr_model, val_df)

0.9386522823720056

In [25]:
rocauc(lr_model, test_df)

0.7000027547230876

In [26]:
ne(lr_model, test_df)

0.9220781622206322

## Make submission

Join the [competition](https://www.kaggle.com/c/mlbd-20-ctr-prediction-1) and make a submission

In [198]:
TEST_PATH = os.path.join(DATA_PATH, 'test.csv')
test_df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TEST_PATH)

In [199]:
test_df.show(1)

+----+---+---+---+----+---+---+---+---+----+----+----+----+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+----+----+--------+----+--------+--------+----+----+------------+
| _c1|_c2|_c3|_c4| _c5|_c6|_c7|_c8|_c9|_c10|_c11|_c12|_c13|    _c14|    _c15|    _c16|    _c17|    _c18|    _c19|    _c20|    _c21|    _c22|    _c23|    _c24|    _c25|    _c26|    _c27|    _c28|    _c29|    _c30|    _c31|_c32|_c33|    _c34|_c35|    _c36|    _c37|_c38|_c39|          id|
+----+---+---+---+----+---+---+---+---+----+----+----+----+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+----+----+--------+----+--------+--------+----+----+------------+
|null| 19|  2|  4|4576|  6|  6|  5| 15|null|   2|null|   4|05db9164|b961056b|bc015839|4120ff84|4cf72387|6f6d9be8|2682a57d|5b392875|a73ee510

In [200]:
test_df = test_df.fillna(0, subset=num_columns)
test_df = pipeline_model.transform(test_df)

# we don't need any other colums anymore
test_df = test_df.select(['id', 'features'])
test_df.show(1)

+------------+--------------------+
|          id|            features|
+------------+--------------------+
|566935904713|(1960,[1,2,3,4,5,...|
+------------+--------------------+
only showing top 1 row



In [201]:
test_predictions = lr_model.transform(test_df)

In [202]:
test_predictions.show(2)

+------------+--------------------+--------------------+--------------------+----------+
|          id|            features|       rawPrediction|         probability|prediction|
+------------+--------------------+--------------------+--------------------+----------+
|566935904713|(1960,[1,2,3,4,5,...|[1.15225048931791...|[0.75992173791025...|       0.0|
|566935904715|(1960,[1,2,4,7,8,...|[1.02031261382024...|[0.73503348837590...|       0.0|
+------------+--------------------+--------------------+--------------------+----------+
only showing top 2 rows



In [208]:
subdf = test_predictions.select(['id', 'probability']).rdd.map(lambda r: (r[0], float(r[1][1]))).toDF(['id', 'proba'])

In [209]:
subdf.show(5)

+------------+-------------------+
|          id|              proba|
+------------+-------------------+
|566935904713|0.24007826208974567|
|566935904715|0.26496651162409074|
|566935904727| 0.2719769850690206|
|566935904737| 0.3796737742222886|
|566935904741| 0.2572364387129162|
+------------+-------------------+
only showing top 5 rows



In [214]:
subdf.repartition(1).write.csv('/workspace/data/criteo_lr_submission.csv')